In [23]:
import pandas as pd
import numpy as np
import warnings 
warnings.filterwarnings('ignore')

# 파일 불러오기
df_1 = pd.read_excel("../데이터자료/최신데이터반영.xlsx")    

# = df[['1Y_대비_irs', "1Y_대비_crs", "대비_국고10년", '대비_ndf']]
df_1.columns


Index(['DateTime', '대비_irs_1Y', '대비_irs_2Y', '대비_irs_3Y', '대비_irs_5Y',
       '대비_irs_10Y', '대비_crs_1Y', '대비_crs_2Y', '대비_crs_3Y', '대비_crs_5Y',
       '대비_crs_10Y', '대비_swapbasis_1Y', '대비_swapbasis_2Y', '대비_swapbasis_3Y',
       '대비_swapbasis_5Y', '대비_swapbasis_10Y', '대비_국고_1Y', '대비_국고_3Y',
       '대비_국고_5Y', '대비_국고_10Y', '대비_통안_1Y', '대비_통안_2Y', '대비_ndf', '스왑포인트_1M',
       '전일종가_ex', '종가_ex', '종가_NDF_차이', '베이시스_1Y', '베이시스_2Y', '베이시스_3Y',
       '베이시스_5Y', '베이시스_10Y', 'Swap Point', 'ndf_mid', '국고1년', '국고3년', '국고5년',
       '국고10년', '통안364일', '종가_ex.1', '종가'],
      dtype='object')

In [28]:
x = df_1[["대비_irs_1Y", "대비_crs_1Y", "대비_국고_10Y","대비_ndf"]]

x.columns =[["1Y_대비_irs", "1Y_대비_crs", "대비_국고10년", "대비_ndf"]]
x = x.dropna()


대비_ndf   -9.5
Name: 1, dtype: float64

In [30]:
for i in x.index:
    if x.loc[i,"대비_ndf"] >= 0: 
        #irs crs <0 이므로 상승 국고 > 0 이므로 상승 NDF도 상승
        if x.loc[i,"1Y_대비_irs"] < 0 & x.loc[i,"1Y_대비_crs"] < 0  & x.loc[i,"대비_국고10년"] >= 0 :
            x.loc[i,"lable"] = 1
            
        #irs crs <0 상승 국고 < 0 하락 추세                
        elif x.loc[i,"1Y_대비_irs"] < 0 & x.loc[i,"1Y_대비_crs"]  < 0 & x.loc[i,"대비_국고10년"] < 0 :
            # 값들의 합이 - 이므로 irs 상승 추세가 더 강함  
            if x.loc[i,"1Y_대비_irs"] + x.loc[i,"1Y_대비_crs"] - x.loc[i,"대비_국고10년"]  <= 0 : 
                x.loc[i,"lable"] = 1
            # 값들의 합이 - 이므로 상승 추세가 더 강함 국고채의 하락추세가 강함
            elif x.loc[i,"1Y_대비_irs"] + x.loc[i,"1Y_대비_crs"] - x.loc[i,"대비_국고10년"]  > 0 : 
                # ndf 값이 절대값 평균의 값보다 작으면 영향력이 이자율보다 작다고 판단 값이 크면 크다고 판단 
                if abs(x.loc[i,"대비_ndf"]) <= abs(x["대비_ndf"]).mean():
                    x.loc[i,"lable"] = 0
                elif abs(x.loc[i,"대비_ndf"]) > abs(x["대비_ndf"]).mean():
                    x.loc[i,"lable"] = 1
        
        #irs <0 상승추제 crs == 0 국고 > 0 상승추세이므로 상승            
        elif x.loc[i,"1Y_대비_irs"] < 0 & x.loc[i,"1Y_대비_crs"] == 0  & x.loc[i,"대비_국고10년"] > 0 :
            if abs(x.loc[i,"대비_ndf"]) <= abs(x["대비_ndf"]).mean():
                x.loc[i,"lable"] = 0
            elif abs(x.loc[i,"대비_ndf"]) > abs(x["대비_ndf"]).mean():
                x.loc[i,"lable"] = 1
            
                    
        #irs < 0 상승 crs == 0 국고 < 0하락추세                  
        elif x.loc[i,"1Y_대비_irs"] < 0 & x.loc[i,"1Y_대비_crs"] == 0 & x.loc[i,"대비_국고10년"] <= 0 :
            # irs가 더 크므로 상승추세
            if x.loc[i,"1Y_대비_irs"] - x.loc[i,"대비_국고10년"]  <= 0:
                if abs(x.loc[i,"대비_ndf"]) <= abs(x["대비_ndf"]).mean():
                    x.loc[i,"lable"] = 0
                elif abs(x.loc[i,"대비_ndf"]) > abs(x["대비_ndf"]).mean():
                    x.loc[i,"lable"] = 1
            # 국고채가 더 큼으로 하락ㅇ추세
            elif x.loc[i,"1Y_대비_irs"] - x.loc[i,"대비_국고10년"]  > 0:
                x.loc[i,"lable"] = 0
                                    
        #irs <0 상승추세 crs > 0 하락 추세 국고 > 0 상승 추세
        elif x.loc[i,"1Y_대비_irs"] < 0 & x.loc[i,"1Y_대비_crs"] > 0  & x.loc[i,"대비_국고10년"] >= 0 :
            # irs의 값이 더 크므로 상승추세
            if x.loc[i,"1Y_대비_irs"] + x.loc[i,"1Y_대비_crs"]  <= 0:
                x.loc[i,"lable"] = 1
            # crs의 값이 더 크므로 하락추세    
            elif x.loc[i,"1Y_대비_irs"] + x.loc[i,"1Y_대비_crs"]  > 0:
                # crs보다 국고채의 값이 더 큼으로 상승추세
                if x.loc[i,"1Y_대비_irs"] + x.loc[i,"1Y_대비_crs"] - x.loc[i,"대비_국고10년"] <= 0 :
                    x.loc[i,"lable"] = 1 
                # crs가 국고채의 값이 더 큼으로 하락추세
                elif x.loc[i,"1Y_대비_irs"] + x.loc[i,"1Y_대비_crs"] - x.loc[i,"대비_국고10년"] > 0 :    
                     # 값 절대ndf 값이 절대값 평균의 값보다 작으면 영향력이 이자율보다 작다고 판단 값이 크면 크다고 판단 
                    if abs(x.loc[i,"대비_ndf"]) <= abs(x["대비_ndf"]).mean():
                        x.loc[i,"lable"] = 0
                    elif abs(x.loc[i,"대비_ndf"]) > abs(x["대비_ndf"]).mean():
                        x.loc[i,"lable"] = 1
                             
        #irs <0 상승추세 crs > 0 하락 추세 국고 < 0 하락 추세
        elif x.loc[i,"1Y_대비_irs"] < 0 & x.loc[i,"1Y_대비_crs"] > 0 & x.loc[i,"대비_국고10년"] < 0 :
            # crs가 irs보다 더 큼으로 하락추세
            if x.loc[i,"1Y_대비_irs"] + x.loc[i,"1Y_대비_crs"] >= 0:
                # 값 절대ndf 값이 절대값 평균의 값보다 작으면 영향력이 이자율보다 작다고 판단 값이 크면 크다고 판단 
                if abs(x.loc[i,"대비_ndf"]) <= abs(x["대비_ndf"]).mean():
                    x.loc[i,"lable"] = 0
                elif abs(x.loc[i,"대비_ndf"]) > abs(x["대비_ndf"]).mean():
                    x.loc[i,"lable"] = 1
             # crs보다 irs가 더 큼으로 상승추세
            elif x.loc[i,"1Y_대비_irs"] + x.loc[i,"1Y_대비_crs"] < 0: 
                # irs보다 국고채가 더 큼으로 하락추세
                if x.loc[i,"1Y_대비_irs"] + x.loc[i,"대비_국고10년"] - x.loc[i,"대비_국고10년"] > 0 : 
                    # 값 절대ndf 값이 절대값 평균의 값보다 작으면 영향력이 이자율보다 작다고 판단 값이 크면 크다고 판단
                    if abs(x.loc[i,"대비_ndf"]) <= abs(x["대비_ndf"]).mean():   
                        x.loc[i,"lable"] = 0    
                    elif abs(x.loc[i,"대비_ndf"]) > abs(x["대비_ndf"]).mean():  
                        x.loc[i,"lable"] = 1
                # irs가 국고채보다 더 큼으로 상승추세
                elif x.loc[i,"1Y_대비_irs"] + x.loc[i,"대비_국고10년"] - x.loc[i,"대비_국고10년"] <= 0 :
                    x.loc[i,"lable"] = 1 
        
        #irs =00  crs < 0 상승 추세 국고 > 0 상승 추세
        elif x.loc[i,"1Y_대비_irs"] == 0 & x.loc[i,"1Y_대비_crs"] < 0  & x.loc[i,"대비_국고10년"] >= 0 :
            x.loc[i,"lable"] = 1
            
        #irs =00  crs < 0 상승 추세 국고 < 0 하락 추세             
        elif x.loc[i,"1Y_대비_irs"] == 0 & x.loc[i,"1Y_대비_crs"]  < 0 & x.loc[i,"대비_국고10년"] < 0 :
            # crs가 국고채의 값이 더 큼으로 상승추세
            if x.loc[i,"1Y_대비_crs"] - x.loc[i,"대비_국고10년"]  <= 0 : 
                x.loc[i,"lable"] = 1
            # crs보다 국고채의 값이 더 큼으로 하락추세    
            elif x.loc[i,"1Y_대비_crs"] - x.loc[i,"대비_국고10년"]  > 0 :
                # 값 절대ndf 값이 절대값 평균의 값보다 작으면 영향력이 이자율보다 작다고 판단 값이 크면 크다고 판단  
                if abs(x.loc[i,"대비_ndf"]) <= abs(x["대비_ndf"]).mean():
                    x.loc[i,"lable"] = 0
                elif abs(x.loc[i,"대비_ndf"]) > abs(x["대비_ndf"]).mean():
                    x.loc[i,"lable"] = 1
                    
        # 국고채가 0보다 큼으로 상승추세           
        elif x.loc[i,"1Y_대비_irs"] == 0 & x.loc[i,"1Y_대비_crs"] == 0  & x.loc[i,"대비_국고10년"] >= 0 :
            x.loc[i,"lable"] = 1
            
        # 국고채가 0보다 큼으로 하락추세                  
        elif x.loc[i,"1Y_대비_irs"] == 0 & x.loc[i,"1Y_대비_crs"] == 0 & x.loc[i,"대비_국고10년"] < 0 :
            # 값 절대ndf 값이 절대값 평균의 값보다 작으면 영향력이 이자율보다 작다고 판단 값이 크면 크다고 판단
            if abs(x.loc[i,"대비_ndf"]) <= abs(x["대비_ndf"]).mean():
                x.loc[i,"lable"] = 0
            elif abs(x.loc[i,"대비_ndf"]) > abs(x["대비_ndf"]).mean():
                x.loc[i,"lable"] = 1
        
        #irs ==0 crs 하락추세 국고채 상승추세
        elif x.loc[i,"1Y_대비_irs"] == 0 & x.loc[i,"1Y_대비_crs"] > 0  & x.loc[i,"대비_국고10년"] >= 0 :
            # crs보다 국고채가 더큼으로 상승추세
            if x.loc[i,"1Y_대비_crs"] - x.loc[i,"대비_국고10년"] <= 0:
                x.loc[i,"lable"] = 1
            # crs가 국고보다 더큼으로 하락추세    
            if x.loc[i,"1Y_대비_crs"] + x.loc[i,"대비_국고10년"] > 0:
                #ndf 절대값이 ndf의 절대값 평균의 값보다 작으면 영향력이 이자율보다 작다고 판단 값이 크면 크다고 판단
                if abs(x.loc[i,"대비_ndf"]) <= abs(x["대비_ndf"]).mean():
                    x.loc[i,"lable"] = 0
                elif abs(x.loc[i,"대비_ndf"]) > abs(x["대비_ndf"]).mean():
                    x.loc[i,"lable"] = 1               
        
        #irs ==0 crs 하락추세 국고채 하락추세
        elif x.loc[i,"1Y_대비_irs"] == 0 & x.loc[i,"1Y_대비_crs"] > 0 & x.loc[i,"대비_국고10년"] < 0 :
            #ndf 절대값이 ndf의 절대값 평균의 값보다 작으면 영향력이 이자율보다 작다고 판단 값이 크면 크다고 판단    
            if abs(x.loc[i,"대비_ndf"]) <= abs(x["대비_ndf"]).mean():
                    x.loc[i,"lable"] = 0
            elif abs(x.loc[i,"대비_ndf"]) > abs(x["대비_ndf"]).mean():
                    x.loc[i,"lable"] = 1
        
        #irs 하락 crs 상승추세 국고채 상승추세            
        elif x.loc[i,"1Y_대비_irs"] > 0 & x.loc[i,"1Y_대비_crs"] < 0  & x.loc[i,"대비_국고10년"] >= 0 :
            # crs가 irs보다 큼으로 상승추세
            if x.loc[i,"1Y_대비_irs"] + x.loc[i,"1Y_대비_crs"]  <= 0:
                x.loc[i,"lable"] = 1
            # irs가 crs보다 큼으로 하락추세
            elif x.loc[i,"1Y_대비_irs"] + x.loc[i,"1Y_대비_crs"] > 0:
                # irs보다 국고채가 더 큼으로 상승추세
                if x.loc[i,"1Y_대비_irs"] + x.loc[i,"1Y_대비_crs"] - x.loc[i,"대비_국고10년"] <= 0 :
                    x.loc[i,"lable"] = 1 
                # irs가 국고채보다 더 큼으로 하락추세
                elif x.loc[i,"1Y_대비_irs"] + x.loc[i,"1Y_대비_crs"] - x.loc[i,"대비_국고10년"] > 0 :
                    #ndf 절대값이 ndf의 절대값 평균의 값보다 작으면 영향력이 이자율보다 작다고 판단 값이 크면 크다고 판단    
                    if abs(x.loc[i,"대비_ndf"]) <= abs(x["대비_ndf"]).mean():
                        x.loc[i,"lable"] = 0
                    elif abs(x.loc[i,"대비_ndf"]) > abs(x["대비_ndf"]).mean():
                        x.loc[i,"lable"] = 1
                        
        #irs 하락 crs 상승추세 국고채 하락추세                     
        elif x.loc[i,"1Y_대비_irs"] > 0 & x.loc[i,"1Y_대비_crs"] < 0 & x.loc[i,"대비_국고10년"] < 0 :
            # crs가 더 크므로 상승추세
            if x.loc[i,"1Y_대비_irs"] + x.loc[i,"1Y_대비_crs"] <= 0 : 
                # crs가 더 크므로 상승추세
               if x.loc[i,"1Y_대비_irs"] + x.loc[i,"1Y_대비_crs"] - x.loc[i,"대비_국고10년"] <= 0 :
                   x.loc[i,"lable"] = 1
                # 국고체가 더 크므로 하락추세
               elif x.loc[i,"1Y_대비_irs"] + x.loc[i,"1Y_대비_crs"] - x.loc[i,"대비_국고10년"] > 0 :
                    if abs(x.loc[i,"대비_ndf"]) <= abs(x["대비_ndf"]).mean():
                        x.loc[i,"lable"] = 0
                    elif abs(x.loc[i,"대비_ndf"]) > abs(x["대비_ndf"]).mean():
                        x.loc[i,"lable"] = 1
            # irs가 더크므로 하락추세        
            elif x.loc[i,"1Y_대비_irs"] + x.loc[i,"1Y_대비_crs"] > 0 : 
                if abs(x.loc[i,"대비_ndf"]) <= abs(x["대비_ndf"]).mean():
                    x.loc[i,"lable"] = 0
                elif abs(x.loc[i,"대비_ndf"]) > abs(x["대비_ndf"]).mean():
                    x.loc[i,"lable"] = 1
                    
        #irs 하락추세 국고채 상승 추세
        elif x.loc[i,"1Y_대비_irs"] > 0 & x.loc[i,"1Y_대비_crs"] == 0  & x.loc[i,"대비_국고10년"] >= 0 :
            #국고채가 더 크므로 상승추세
            if x.loc[i,"1Y_대비_irs"] - x.loc[i,"대비_국고10년"] <= 0 :
                x.loc[i,"lable"] = 1
            # irs가 더 크므로 하락추세
            elif x.loc[i,"1Y_대비_irs"] - x.loc[i,"대비_국고10년"] > 0 :
                #ndf 절대값이 ndf의 절대값 평균의 값보다 작으면 영향력이 이자율보다 작다고 판단 값이 크면 크다고 판단    
                if abs(x.loc[i,"대비_ndf"]) <= abs(x["대비_ndf"]).mean():
                    x.loc[i,"lable"] = 0
                elif abs(x.loc[i,"대비_ndf"]) > abs(x["대비_ndf"]).mean():
                    x.loc[i,"lable"] = 1
                
                
        # irs 하락추세 국고채 하락추세             
        elif x.loc[i,"1Y_대비_irs"] > 0 & x.loc[i,"1Y_대비_crs"] == 0 & x.loc[i,"대비_국고10년"] < 0 :
            #ndf 절대값이 ndf의 절대값 평균의 값보다 작으면 영향력이 이자율보다 작다고 판단 값이 크면 크다고 판단
            if abs(x.loc[i,"대비_ndf"]) <= abs(x["대비_ndf"]).mean():
                x.loc[i,"lable"] = 0
            elif abs(x.loc[i,"대비_ndf"]) > abs(x["대비_ndf"]).mean():
                x.loc[i,"lable"] = 1
                
        #irs 하락 crs 하락추세 국고채 상승추세        
        elif x.loc[i,"1Y_대비_irs"] > 0 & x.loc[i,"1Y_대비_crs"]  > 0  & x.loc[i,"대비_국고10년"] > 0 :
            #국고채가 크므로 상승추세
            if x.loc[i,"1Y_대비_irs"] + x.loc[i,"1Y_대비_crs"] - x.loc[i,"대비_국고10년"] <= 0 :
                x.loc[i,"lable"] = 1
            #irs가 크므루 하락추세
            elif x.loc[i,"1Y_대비_irs"] + x.loc[i,"1Y_대비_crs"] - x.loc[i,"대비_국고10년"] > 0 :
                #ndf 절대값이 ndf의 절대값 평균의 값보다 작으면 영향력이 이자율보다 작다고 판단 값이 크면 크다고 판단
                if abs(x.loc[i,"대비_ndf"]) <= abs(x["대비_ndf"]).mean():
                    x.loc[i,"lable"] = 0
                elif abs(x.loc[i,"대비_ndf"]) > abs(x["대비_ndf"]).mean():
                    x.loc[i,"lable"] = 1              
       
        #irs 하락 crs 하락추세 국고채 하락추세
        elif x.loc[i,"1Y_대비_irs"] > 0 & x.loc[i,"1Y_대비_crs"]  > 0  & x.loc[i,"대비_국고10년"] <= 0 :
            #ndf 절대값이 ndf의 절대값 평균의 값보다 작으면 영향력이 이자율보다 작다고 판단 값이 크면 크다고 판단
            if abs(x.loc[i,"대비_ndf"]) <= abs(x["대비_ndf"]).mean():
                x.loc[i,"lable"] = 1
            elif abs(x.loc[i,"대비_ndf"]) > abs(x["대비_ndf"]).mean():
                x.loc[i,"lable"] = 0
            
                     
  
        

ValueError: The truth value of a Series is ambiguous. Use a.empty, a.bool(), a.item(), a.any() or a.all().

In [16]:
for i in x.index:    
    ####### ndf < 0 하락압력                                         
    if x.loc[i,"대비_ndf"] < 0: 
        #irs 상승 crs 상승 <0  국고 > 0 상승
        if x.loc[i,"1Y_대비_irs"] < 0 and x.loc[i,"1Y_대비_crs"] < 0  and x.loc[i,"대비_국고10년"] >= 0 :
            #ndf 절대값이 ndf의 절대값 평균의 값보다 작으면 영향력이 이자율보다 작다고 판단 값이 크면 크다고 판단
            if abs(x.loc[i,"대비_ndf"]) <= abs(x["대비_ndf"]).mean():
                x.loc[i,"lable"] = 1
            elif abs(x.loc[i,"대비_ndf"]) > abs(x["대비_ndf"]).mean():
                x.loc[i,"lable"] = 0
                
        #irs 상승 crs 상승   국고 하락        
        elif x.loc[i,"1Y_대비_irs"] < 0 and x.loc[i,"1Y_대비_crs"]  < 0 and x.loc[i,"대비_국고10년"] < 0 :
            # irs쪽이 더크므로 상승압력
            if x.loc[i,"1Y_대비_irs"] + x.loc[i,"1Y_대비_crs"] + x.loc[i,"대비_국고10년"]  <= 0 : 
                 #ndf 절대값이 ndf의 절대값 평균의 값보다 작으면 영향력이 이자율보다 작다고 판단 값이 크면 크다고 판단
                if abs(x.loc[i,"대비_ndf"]) <= abs(x["대비_ndf"]).mean():
                    x.loc[i,"lable"] = 1
                elif abs(x.loc[i,"대비_ndf"]) > abs(x["대비_ndf"]).mean():
                    x.loc[i,"lable"] = 0
            # 국고채쪽이 더크므로 하락압력
            elif x.loc[i,"1Y_대비_irs"] + x.loc[i,"1Y_대비_crs"] - x.loc[i,"대비_국고10년"]  > 0 : 
                x.loc[i,"lable"] = 0
                
        #irs 상승 국고채 상승         
        elif x.loc[i,"1Y_대비_irs"] < 0 and x.loc[i,"1Y_대비_crs"] == 0  and x.loc[i,"대비_국고10년"] >= 0 :
            #ndf 절대값이 ndf의 절대값 평균의 값보다 작으면 영향력이 이자율보다 작다고 판단 값이 크면 크다고 판단
            if abs(x.loc[i,"대비_ndf"]) <= abs(x["대비_ndf"]).mean():
                x.loc[i,"lable"] = 1
            elif abs(x.loc[i,"대비_ndf"]) > abs(x["대비_ndf"]).mean():
                x.loc[i,"lable"] = 0
        
        #irs 상승 국고채 하락                
        elif x.loc[i,"1Y_대비_irs"] < 0 and x.loc[i,"1Y_대비_crs"] == 0 and x.loc[i,"대비_국고10년"] < 0 :
            # irs쪽이 더크므로 상승압력
            if x.loc[i,"1Y_대비_irs"] - x.loc[i,"대비_국고10년"]  <= 0 : 
                #ndf 절대값이 ndf의 절대값 평균의 값보다 작으면 영향력이 이자율보다 작다고 판단 값이 크면 크다고 판단
                if abs(x.loc[i,"대비_ndf"]) <= abs(x["대비_ndf"]).mean():
                    x.loc[i,"lable"] = 1
                elif abs(x.loc[i,"대비_ndf"]) > abs(x["대비_ndf"]).mean():
                    x.loc[i,"lable"] = 0
            # 국고채쪽이 더크므로 하락압력
            elif x.loc[i,"1Y_대비_irs"] - x.loc[i,"대비_국고10년"]  > 0 : 
                x.loc[i,"lable"] = 0
                    
        #irs 상승 crs 하락 국고 상승
        elif x.loc[i,"1Y_대비_irs"] < 0 and x.loc[i,"1Y_대비_crs"] > 0  and x.loc[i,"대비_국고10년"] > 0 :
            # irs가 더크므로 상승효과큼 
            if x.loc[i,"1Y_대비_irs"] + x.loc[i,"1Y_대비_crs"]  <= 0:
                #ndf 절대값이 ndf의 절대값 평균의 값보다 작으면 영향력이 이자율보다 작다고 판단 값이 크면 크다고 판단
                if abs(x.loc[i,"대비_ndf"]) <= abs(x["대비_ndf"]).mean():
                    x.loc[i,"lable"] = 1
                elif abs(x.loc[i,"대비_ndf"]) > abs(x["대비_ndf"]).mean():
                    x.loc[i,"lable"] = 0
            # crs가 더크므로 하락효과          
            elif x.loc[i,"1Y_대비_irs"] + x.loc[i,"1Y_대비_crs"] > 0:
                # 국고채가 큼으로 상승효과
                if x.loc[i,"1Y_대비_irs"] + x.loc[i,"1Y_대비_crs"] - x.loc[i,"대비_국고10년"] <= 0 :
                     #ndf 절대값이 ndf의 절대값 평균의 값보다 작으면 영향력이 이자율보다 작다고 판단 값이 크면 크다고 판단
                    if abs(x.loc[i,"대비_ndf"]) <= abs(x["대비_ndf"]).mean():
                        x.loc[i,"lable"] = 1
                    elif abs(x.loc[i,"대비_ndf"]) > abs(x["대비_ndf"]).mean():
                        x.loc[i,"lable"] = 0 
                # crs가 큼으로 하락효과
                elif x.loc[i,"1Y_대비_irs"] + x.loc[i,"1Y_대비_crs"] - x.loc[i,"대비_국고10년"] > 0 :    
                    x.loc[i,"lable"] = 0
          
        # irs 상승 crs 하락 국고채 하락효과
        elif x.loc[i,"1Y_대비_irs"] < 0 and x.loc[i,"1Y_대비_crs"] > 0 and x.loc[i,"대비_국고10년"] <= 0 :
            # irs더 큼으로 상승효과  
            if x.loc[i,"1Y_대비_irs"] + x.loc[i,"1Y_대비_crs"] <= 0:
                # irs가 큼으로 상승효과
                if x.loc[i,"1Y_대비_irs"] + x.loc[i,"1Y_대비_crs"] - x.loc[i,"대비_국고10년"] <= 0 :
                    #ndf 절대값이 ndf의 절대값 평균의 값보다 작으면 영향력이 이자율보다 작다고 판단 값이 크면 크다고 판단
                    if abs(x.loc[i,"대비_ndf"]) <= abs(x["대비_ndf"]).mean():
                        x.loc[i,"lable"] = 1
                    elif abs(x.loc[i,"대비_ndf"]) > abs(x["대비_ndf"]).mean():
                        x.loc[i,"lable"] = 0
                #국고채가 더 큼으로 하락압력
                elif x.loc[i,"1Y_대비_irs"] + x.loc[i,"1Y_대비_crs"] - x.loc[i,"대비_국고10년"] > 0 :    
                        x.loc[i,"lable"] = 0
            # crs가 더 큼으로 하락효과
            elif  x.loc[i,"1Y_대비_irs"] + x.loc[i,"1Y_대비_crs"] > 0 : 
                x.loc[i,"lable"] = 0
                
        # crs 상승 국고채 상승
        elif x.loc[i,"1Y_대비_irs"] == 0 and x.loc[i,"1Y_대비_crs"] < 0  and x.loc[i,"대비_국고10년"] > 0 :
            #ndf 절대값이 ndf의 절대값 평균의 값보다 작으면 영향력이 이자율보다 작다고 판단 값이 크면 크다고 판단
            if abs(x.loc[i,"대비_ndf"]) <= abs(x["대비_ndf"]).mean():
                x.loc[i,"lable"] = 0
            elif abs(x.loc[i,"대비_ndf"]) > abs(x["대비_ndf"]).mean():
                x.loc[i,"lable"] = 1            
        
        # crs 상승 국고채 하락
        elif x.loc[i,"1Y_대비_irs"] == 0 and x.loc[i,"1Y_대비_crs"]  < 0 and x.loc[i,"대비_국고10년"] <= 0 :
            # crs가 큼으로 상승효과 
            if x.loc[i,"1Y_대비_crs"] - x.loc[i,"대비_국고10년"]  <= 0 : 
                #ndf 절대값이 ndf의 절대값 평균의 값보다 작으면 영향력이 이자율보다 작다고 판단 값이 크면 크다고 판단
                if abs(x.loc[i,"대비_ndf"]) <= abs(x["대비_ndf"]).mean():
                    x.loc[i,"lable"] = 1
                elif abs(x.loc[i,"대비_ndf"]) > abs(x["대비_ndf"]).mean():
                    x.loc[i,"lable"] = 0
            # 국고채가 큼으로 하락압력
            elif x.loc[i,"1Y_대비_crs"] - x.loc[i,"대비_국고10년"]  > 0 : 
                x.loc[i,"lable"] = 0
                
        # 국고채 상승압력          
        elif x.loc[i,"1Y_대비_irs"] == 0 and x.loc[i,"1Y_대비_crs"] == 0  and x.loc[i,"대비_국고10년"] > 0 :
            #ndf 절대값이 ndf의 절대값 평균의 값보다 작으면 영향력이 이자율보다 작다고 판단 값이 크면 크다고 판단
            if abs(x.loc[i,"대비_ndf"]) <= abs(x["대비_ndf"]).mean():
                x.loc[i,"lable"] = 1
            elif abs(x.loc[i,"대비_ndf"]) > abs(x["대비_ndf"]).mean():
                x.loc[i,"lable"] = 0            
        
        # 국고채 하락압력
        elif x.loc[i,"1Y_대비_irs"] == 0 and x.loc[i,"1Y_대비_crs"] == 0 and x.loc[i,"대비_국고10년"] <= 0 :
            x.loc[i,"lable"] = 0
        
        # crs 하락 국고채 상승
        elif x.loc[i,"1Y_대비_irs"] == 0 and x.loc[i,"1Y_대비_crs"] > 0  and x.loc[i,"대비_국고10년"] > 0 :
            # 국고채가 더 크므로 상승압력 
            if x.loc[i,"1Y_대비_crs"] - x.loc[i,"대비_국고10년"] < 0:
                #ndf 절대값이 ndf의 절대값 평균의 값보다 작으면 영향력이 이자율보다 작다고 판단 값이 크면 크다고 판단
                if abs(x.loc[i,"대비_ndf"]) <= abs(x["대비_ndf"]).mean():
                    x.loc[i,"lable"] = 1
                elif abs(x.loc[i,"대비_ndf"]) > abs(x["대비_ndf"]).mean():
                    x.loc[i,"lable"] = 0 
            
            # crs가 더 크므로 하락압력
            elif x.loc[i,"1Y_대비_crs"] - x.loc[i,"대비_국고10년"] >= 0:
                x.loc[i,"lable"] = 0
        
        # crs 하락 국고채 하락압력                    
        elif x.loc[i,"1Y_대비_irs"] == 0 and x.loc[i,"1Y_대비_crs"] > 0 and x.loc[i,"대비_국고10년"] <= 0 :
                x.loc[i,"lable"] = 0
        
        # irs 하락 crs 상승 국고채 상승
        elif x.loc[i,"1Y_대비_irs"] > 0 and x.loc[i,"1Y_대비_crs"] < 0  and x.loc[i,"대비_국고10년"] > 0 :
            # crs가 더 크므로 상승압력
            if x.loc[i,"1Y_대비_irs"] + x.loc[i,"1Y_대비_crs"]  <= 0:
                if abs(x.loc[i,"대비_ndf"]) <= abs(x["대비_ndf"]).mean():
                    x.loc[i,"lable"] = 1
                elif abs(x.loc[i,"대비_ndf"]) > abs(x["대비_ndf"]).mean():
                    x.loc[i,"lable"] = 0
            # irs가 더 크므로 하락압력
            elif x.loc[i,"1Y_대비_irs"] + x.loc[i,"1Y_대비_crs"] > 0:
                # 국고채가 더크므로 상승압력
                if x.loc[i,"1Y_대비_irs"] + x.loc[i,"1Y_대비_crs"] - x.loc[i,"대비_국고10년"] <= 0 :
                    #ndf 절대값이 ndf의 절대값 평균의 값보다 작으면 영향력이 이자율보다 작다고 판단 값이 크면 크다고 판단
                    if abs(x.loc[i,"대비_ndf"]) <= abs(x["대비_ndf"]).mean():
                        x.loc[i,"lable"] = 1
                    elif abs(x.loc[i,"대비_ndf"]) > abs(x["대비_ndf"]).mean():
                        x.loc[i,"lable"] = 0 
                # irs가 더 크므로 하락압력
                elif x.loc[i,"1Y_대비_irs"] + x.loc[i,"1Y_대비_crs"] - x.loc[i,"대비_국고10년"] > 0 :    
                    x.loc[i,"lable"] = 0
                    
        # irs 하락 crs 상승 국고채 하락                  
        elif x.loc[i,"1Y_대비_irs"] > 0 and x.loc[i,"1Y_대비_crs"] < 0 and x.loc[i,"대비_국고10년"] <= 0 :
            # crs가 더 크므로 상승압력
            if x.loc[i,"1Y_대비_irs"] + x.loc[i,"대비_국고10년"]  <= 0 : 
                #crs 가 더크므로 상승압력
                if x.loc[i,"1Y_대비_irs"] + x.loc[i,"대비_국고10년"] - x.loc[i,"대비_국고10년"] <= 0 : 
                    if abs(x.loc[i,"대비_ndf"]) <= abs(x["대비_ndf"]).mean():
                        x.loc[i,"lable"] = 1
                    elif abs(x.loc[i,"대비_ndf"]) > abs(x["대비_ndf"]).mean():
                        x.loc[i,"lable"] = 0
                #국고채가 크므로 하락압력
                elif x.loc[i,"1Y_대비_irs"] + x.loc[i,"대비_국고10년"] - x.loc[i,"대비_국고10년"] > 0 : 
                    x.loc[i,"lable"] = 0
            # irs 가 더 크므로 하락압력
            elif x.loc[i,"1Y_대비_irs"] + x.loc[i,"대비_국고10년"] > 0 : 
                x.loc[i,"lable"] = 0
       
        #irs 하락 국고채 상승
        elif x.loc[i,"1Y_대비_irs"] > 0 and x.loc[i,"1Y_대비_crs"] == 0  and x.loc[i,"대비_국고10년"] > 0 :
            # 국고채가 더 크므로 상승압력
            if x.loc[i,"1Y_대비_irs"] - x.loc[i,"대비_국고10년"] <= 0 :
                if abs(x.loc[i,"대비_ndf"]) <= abs(x["대비_ndf"]).mean():
                    x.loc[i,"lable"] = 1
                elif abs(x.loc[i,"대비_ndf"]) > abs(x["대비_ndf"]).mean():
                    x.loc[i,"lable"] = 0 
            # irs가 더 크므로 하락압력
            elif x.loc[i,"1Y_대비_irs"] - x.loc[i,"대비_국고10년"] > 0 :
                    x.loc[i,"lable"] = 0
        
        # irs 하락 국고채 하락 하락압력
        elif x.loc[i,"1Y_대비_irs"] > 0 and x.loc[i,"1Y_대비_crs"] == 0 and x.loc[i,"대비_국고10년"] <= 0 :
                x.loc[i,"lable"] = 0
              
        # irs 하락 crs하락 국고채 상승
        elif x.loc[i,"1Y_대비_irs"] > 0 and x.loc[i,"1Y_대비_crs"]  > 0  and x.loc[i,"대비_국고10년"] > 0 :
            # 국고채가 더 크므로 상승압력
            if x.loc[i,"1Y_대비_irs"] + x.loc[i,"1Y_대비_crs"] - x.loc[i,"대비_국고10년"] <= 0 :
                if abs(x.loc[i,"대비_ndf"]) <= abs(x["대비_ndf"]).mean():
                    x.loc[i,"lable"] = 1
                elif abs(x.loc[i,"대비_ndf"]) > abs(x["대비_ndf"]).mean():
                    x.loc[i,"lable"] = 0
            # irs쪽이 더 크므로 하락압력
            elif x.loc[i,"1Y_대비_irs"] + x.loc[i,"1Y_대비_crs"] - x.loc[i,"대비_국고10년"] > 0 :
                x.loc[i,"lable"] = 0
        
        # irs 하락 crs하락 국고채 하락압력
        elif x.loc[i,"1Y_대비_irs"] > 0 and x.loc[i,"1Y_대비_crs"]  > 0  and x.loc[i,"대비_국고10년"] <= 0 :
            x.loc[i,"lable"] = 0

ValueError: The truth value of a Series is ambiguous. Use a.empty, a.bool(), a.item(), a.any() or a.all().